# Scalability analysis
# Best population size for given problem

The best population size for a given problem instance is searched for using a binary search approach.
A population size is deemed sufficient if a fitness less than the predetermined goal fitness is attained 10 times in a row. If this occurs, search is continued on the first half of the current search space otherwise the latter half is searched.

In [1]:
%matplotlib
import sys
sys.path.append('./codebase')

Using matplotlib backend: Qt5Agg


In [2]:
from cooperative_evolution_optimizers import GrayBoxOptimizer, BlackBoxOptimizer
from differential_evolution import DifferentialEvolution as DE
from evolution_strategies import EvolutionStrategies as ES
from particle_swarm_optimization import ParticleSwarmOptimization as PSO
from particle_swarm_optimization import PSOInteractions
from fitness_functions import FunctionFactory as FF

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [50]:
#Searches for the optimal population size as described in the lecture slides using the bisection method
def optimal_popsize_search(get_population, pop_size_start, pop_size_max, repetitions):
    
    pop_size = pop_size_start
    evaluations = None
    
    while(pop_size <= pop_size_max):
        
        success = True
        
        for _ in range(repetitions):
            
            population = get_population(pop_size)
            
            while(not population.has_converged()):
                population.evolve()
            
            if(not population.get_elite_fitness() <= population._goal_fitness):
                pop_size = 2*pop_size
                success = False
                break
        
        if(success):
            if(pop_size == pop_size_start):
                return pop_size, population.get_evaluations()
            
            return binary_popsize_search(get_population, pop_size/2, pop_size, repetitions)
            
    
    return pop_size_max, float('inf')

#Subroutine for optimal_popsize_search
def binary_popsize_search(get_population, pop_size_min, pop_size_max, repetitions):
    
    pop_size = int((pop_size_min + pop_size_max)/2)
    evaluations = None
    
    while(pop_size != pop_size_min):
        
        for _ in range(repetitions):
            
            population = get_population(pop_size)
            
            while(not population.has_converged()):
                population.evolve()
            
            if(not population.get_elite_fitness() <= population._goal_fitness):
                pop_size_min = pop_size
                break
        
        if(not pop_size_min == pop_size):
            pop_size_max = pop_size
            
        pop_size = int((pop_size_min + pop_size_max)/2)
        evaluations = population.get_evaluations()
        
    return pop_size, evaluations
    

In [24]:
#Returns a function which generates new GBO-CC instances 
def GBO_instances(algorithm, get_function, n_functions, input_size, goal_fitness, max_generations):
    
    functions = [get_function() for _ in range(n_functions)]
    genetic_algorithms = [algorithm for _ in range(n_functions)]
    
    n_variables = input_size*n_functions
    lower_bounds = [-2]*n_variables
    upper_bounds = [3]*n_variables
    
    input_spaces = [list(range(i*input_size, (i+1)*input_size)) for i in range(n_functions)]    
    train_partition = input_spaces
          
    def get_instance(pop_size):
        if(algorithm == DE):
            genetic_algorithm_arguments = [{'population_size':pop_size}] * n_functions
        else:
            genetic_algorithm_arguments = [{'population_size':pop_size,
                                            'interaction':PSOInteractions.NORMAL}] * n_functions
            
        return GrayBoxOptimizer(functions = functions,
                       input_spaces = input_spaces,
                       train_partition = train_partition,
                       lower_bounds = lower_bounds, upper_bounds = upper_bounds,
                       genetic_algorithms = genetic_algorithms,
                       genetic_algorithm_arguments = genetic_algorithm_arguments,
                       max_generations = max_generations,
                       goal_fitness = goal_fitness)
    
    return get_instance


#Returns a function which generates new BBO (no CC) instances 
def BBO_instances(algorithm, get_function, n_functions, input_size, goal_fitness, max_generations):
        
        functions = [get_function() for _ in range(n_functions)]
        n_variables = input_size*n_functions
        
        def target_function(x):
            return sum([functions[i](x[i*input_size:(i+1)*input_size]) for i in range(n_functions)])
        
        
        def get_instance(pop_size):
            if(algorithm == DE):
                return algorithm(fitness_function = target_function,
                                population_size = pop_size,
                                lower_bounds = -2,
                                upper_bounds = 3,
                                genome_length = n_variables,
                                goal_fitness = goal_fitness,
                                max_generations = max_generations)
            else:
                return algorithm(fitness_function = target_function,
                population_size = pop_size,
                lower_bounds = -2,
                upper_bounds = 3,
                genome_length = n_variables,
                goal_fitness = goal_fitness,
                max_generations = max_generations,
                interaction = PSOInteractions.NORMAL)
        
        return get_instance

In [ ]:
#The main simulation loop

n_functions_dict = {'sphere': [1,2,4,5,6,7,8,9,10],
                    'rosenbrock': [1,2,3,4,5],
                    'soreb': [1,2]}

goal_fitness_dict = {'sphere': 1e-4,
                    'rosenbrock': 1e-3,
                    'soreb': 1e-3}

input_size_dict = {'sphere': 3,
                    'rosenbrock': 2,
                    'soreb': 2}

max_gen_dict = {'DE': 1000,
               'PSO': 1000}

algo_dict = {'DE': DE, 'PSO': PSO}
func_dict = {'sphere': FF.get_sphere, 'rosenbrock': FF.get_rosenbrock}

repetitions = 10
results = pd.DataFrame(columns = ['optimizer', 'algorithm', 'function', 'n_functions'
                                  ,'population size', 'evaluations'])

for optimizer in ['GBO', 'BBO']:
    for algorithm in ['DE', 'PSO']:
        for function in ['sphere', 'rosenbrock']:
            
            print(optimizer + ',' + algorithm + ',' + function)
            
            n_functions = n_functions_dict[function]
            goal_fitness = goal_fitness_dict[function]
            input_size = input_size_dict[function]
            max_generations = max_gen_dict[algorithm]
            
            for n in n_functions:
                
                for _ in range(repetitions):
                    
                    if(optimizer == 'GBO'):
                        instance_factory = GBO_instances(algo_dict[algorithm], func_dict[function], 
                                                         n, input_size, goal_fitness, max_generations)
                        max_popsize = 500
                    else:
                        instance_factory = BBO_instances(algo_dict[algorithm], func_dict[function], 
                                                         n, input_size, goal_fitness, max_generations)
                        max_popsize = n * 500
                        
                    tmp = optimal_popsize_search(instance_factory, 5, max_popsize, 5)
                    
                    results = results.append({'optimizer': optimizer, 'algorithm': algorithm,
                               'function': function, 'n_functions': n,
                                'population size': tmp[0], 'evaluations': tmp[1]}, ignore_index = True)
                
            results.to_csv('results.csv', index = False)
                

GBO,DE,sphere


In [59]:
instance_factory = BBO_instances(DE, FF.get_rosenbrock, 1, 2, 1e-3, 500)

optimal_popsize_search(instance_factory, 5, 1000, 5)

(80, 1863)